<a href="https://colab.research.google.com/github/swaspicious/ML-Models/blob/main/Building_Support_Vector_Machine_Classifier_from_scratch_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import numpy as np

**Support Vector Machine Classifier**

In [55]:
class SVMclassifier():

  # initiating the hyperparameters (learning rate and no. of iterations)
  def __init__(self, learning_rate, no_of_iterations, lambda_parameter):
    self.learning_rate = learning_rate
    self.no_of_iterations = no_of_iterations
    self.lambda_parameter = lambda_parameter



  # fitting the dataset to the SVM classifier
  def fit(self, x, y):
    self.m, self.n = x.shape    # m--> no. of datapoints in the dataset (rows), n--> no. of input features (no. of columns except the target feature column)
    self.w = np.zeros(self.n)
    self.b = 0
    self.x = x
    self.y = y

    # implementing the gradient descent algorithm for optimization:
    for i in range(self.no_of_iterations):
      self.update_weights()


  # function for updating the model paramters (weight and bias values)
  def update_weights(self):

    # label encoding: (in svm, the model tries to find whether the output is +1 or -1. hence we convert 0 to -1)
    ylabel = np.where(self.y <= 0, -1, 1)  # ylabel represents 'y_i'

    # GRADIENTS:
    for index, x_i in enumerate(self.x):
      condition = ylabel[index] * (np.dot(x_i, self.w) - self.b) >=1  # x_i is basically an array taking all the values of the input features for a particular row.

      if (condition == True):
        dw = 2 * self.lambda_parameter * self.w
        db = 0
      else:
        dw = 2 * self.lambda_parameter * self.w - np.dot(x_i, ylabel[index])
        db = ylabel[index]

      self.w = self.w - self.learning_rate * dw
      self.b = self.b - self.learning_rate * db



  # predicting the label for a given input value (set of input features)
  def predict(self, x):

    output = np.dot(x, self.w) - self.b # this output value can be any positive or negative number
    predicted_labels = np.sign(output)  # np.sign() will round the value to -1 if its negative ( <=1 ) and it will round it to 1 if its positive ( >=1 )

    y_hat = np.where(predicted_labels <= -1, 0, 1) # if the y_i value is -1, we're making it 0 ( since that is the actual value in the dataset i.e. either 0 or 1)

    return y_hat


In [56]:
# model = SVMclassifier(learning_rate = 0.001, no_of_iterations = 1000, lambda_parameter = 0.01) # 'model' is an instance (object) of the class SVMclassifier. It will be represented as 'self' within the class.

In [57]:
# importing the dependencies:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# from svm_classifier import SVMclassifier (importing the class from the saved python file - notepad++)

**Data collection and preprocessing:**

In [58]:
# loading the data from csv file to pandas dataframe:
diabetesdata = pd.read_csv('/content/diabetes (1).csv')

In [59]:
diabetesdata.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [60]:
diabetesdata.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [61]:
# finding the number of rows and columns in this dataset:
diabetesdata.shape

(768, 9)

In [62]:
# getting the statistical measures of the data:
diabetesdata.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [63]:
diabetesdata['Outcome'].value_counts()

,count
Outcome,
0,500
1,268


0 -> non-diabetic people,

1 -> diabetic people


In [64]:
# seperating the features and target:
features = diabetesdata.drop(columns = 'Outcome', axis=1) # note- it is necessary to mention axis=1 while dropping some column.
target = diabetesdata['Outcome']

In [65]:
print(features)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [66]:
print(target)

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


**Data standardization:**

In [67]:
scaler = StandardScaler()

In [68]:
scaler.fit(features)

StandardScaler()

In [69]:
standardized_data = scaler.transform(features)

In [70]:
print(standardized_data)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [71]:
features = standardized_data  # feeding the standardized data back to the 'features' matrix (variable)
target = diabetesdata['Outcome']

In [72]:
print(features)
print(target)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]
0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


**Train Test split:**

In [73]:
xtrain,xtest,ytrain,ytest = train_test_split(features,target,test_size=0.2,random_state=2)

In [74]:
print(features.shape, xtrain.shape, xtest.shape)

(768, 8) (614, 8) (154, 8)


**IMPLEMENTING THE SVM CLASSIFIER i.e. TRAINING THE MODEL**

In [75]:
classifier = SVMclassifier(learning_rate = 0.002, no_of_iterations = 1000, lambda_parameter=0.01)

In [76]:
# training the svm classifier with training data:
classifier.fit(xtrain,ytrain)

**Model Evaluation**

Accuracy score:


In [77]:
xtrain_prediction = classifier.predict(xtrain) # xtrain_prediction is the array of values predicted by our model and we are gonna compare it with the actual values.
accuracy = accuracy_score(ytrain, xtrain_prediction)

In [78]:
print('Accuracy score on training data = ',accuracy)

Accuracy score on training data =  0.7752442996742671


In [79]:
xtest_prediction = classifier.predict(xtest) # xtest_prediction is the array of values predicted by our model and we are gonna compare it with the actual values.
testdata_accuracy = accuracy_score(ytest, xtest_prediction)

In [81]:
print('Accuracy score on test data = ',testdata_accuracy)

Accuracy score on test data =  0.7467532467532467


**Building a predictive system:**

In [88]:
inputdata = (1,103,30,38,83,43.3,0.183,33)

# change the input data to numpy array:
inputdata_as_numpyarray = np.asarray(inputdata)

# reshape the array: (reshaping it coz we'll be making the prediction for a particular row/set of values only)
inputdata_reshaped = inputdata_as_numpyarray.reshape(1,-1)

# standardizing the set of values (input data):
std_data = scaler.transform(inputdata_reshaped)
print(std_data)

prediction = classifier.predict(std_data)
print(prediction)

if(prediction[0] == 0):
   print('The person is not diabetic')
else:
   print('The person is diabetic')

[[-0.84488505 -0.56004775 -2.02166474  1.09545411  0.02778979  1.43512945
  -0.87244072 -0.0204964 ]]
[0]
The person is not diabetic


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
